In [ ]:
%%capture
import os
import pandas as pd
from dj_notebook import activate
from pathlib import Path

# open a tunnel in a shell first if this is a remote DB
# ssh -N my.db.server -L 127.0.0.1:3307:127.0.0.1:3306

env_file = os.environ["META_ENV_REMOTE"]
analysis_folder = Path(os.environ["META_ANALYSIS_FOLDER"])
reports_folder = Path(os.environ["META_ANALYSIS_FOLDER"])
plus = activate(dotenv_file=env_file)
pd.set_option('future.no_silent_downcasting', True)

In [ ]:
from django.apps import apps as django_apps
from edc_export.models_to_file import ModelsToFile
from django.contrib.auth.models import User
from edc_export.constants import CSV
from edc_sites.site import sites

In [ ]:
user = User.objects.get(username="erikvw")
site_ids = list(sites.all().keys())

In [ ]:
study_prefix = "meta"
study_apps = [
    f"{study_prefix}_consent",
    f"{study_prefix}_lists",
    f"{study_prefix}_subject",
    f"{study_prefix}_ae",
    f"{study_prefix}_prn",
    f"{study_prefix}_screening",
]
all_model_names = [
    "edc_appointment.appointment",
    "edc_data_manager.datadictionary",
    "edc_metadata.crfmetadata",
    "edc_metadata.requisitionmetadata",
    "edc_registration.registeredsubject",
    "edc_visit_schedule.subjectschedulehistory",
]

excluded_model_names = ["meta_subject_bloodresultsgludummy"]

In [ ]:
for app_config in django_apps.get_app_configs():
    if app_config.name.startswith(study_prefix):
        if app_config.name in study_apps:
            model_names = [
                model_cls._meta.label_lower
                for model_cls in app_config.get_models()
                if "historical" not in model_cls._meta.label_lower
                and not model_cls._meta.proxy
                and model_cls._meta.label_lower not in excluded_model_names
            ]
            if model_names:
                all_model_names.extend(model_names)

In [ ]:
ModelsToFile(
    user=user,
    models=all_model_names,
    site_ids=site_ids,
    decrypt=False,
    archive_to_single_file=True,
    export_format=CSV
)